In [13]:
import pandas as pd
import numpy as np
import doubleml as dml
from doubleml import DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
np.random.seed(20)

In [32]:
df = pd.read_csv("./data/retention_data (3).csv")
df['treatment'] = df['session_time'].apply(lambda x: 1 if x > 180 else 0)
df.fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15936 entries, 0 to 15935
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           15936 non-null  object 
 1   return            15936 non-null  int64  
 2   total_eqt_earned  15936 non-null  float64
 3   open_crate_count  15936 non-null  float64
 4   total_spent_gem   15936 non-null  float64
 5   revive_gem        15936 non-null  float64
 6   equipment_gem     15936 non-null  float64
 7   total_spent_gold  15936 non-null  float64
 8   daily_shop_gold   15936 non-null  float64
 9   max_stage         15936 non-null  float64
 10  stage_end         15936 non-null  float64
 11  trial_end         15936 non-null  float64
 12  enemy_kill_count  15936 non-null  float64
 13  game_level_max    15936 non-null  float64
 14  pass_5m_lv1       15936 non-null  float64
 15  video_count       15936 non-null  float64
 16  session_time      15936 non-null  float6

In [33]:
potential_confounders = [
    'total_eqt_earned', 'open_crate_count',
    'total_spent_gem', 'equipment_gem', 'total_spent_gold',
    'daily_shop_gold', 'max_stage', 'stage_end', 'trial_end',
    'enemy_kill_count', 'game_level_max'
]

data_dml = dml.DoubleMLData(
    data=df,
    y_col='video_count',
    d_cols='treatment',
    x_cols=potential_confounders,
)

In [34]:
rf_1 = RandomForestRegressor(n_estimators=1000, random_state=0)
rf_2 = RandomForestClassifier(n_estimators=1000, random_state=0)

# DoubleML partially linear regression model
dml_plr = DoubleMLPLR(
    obj_dml_data=data_dml,
    ml_l=rf_1,
    ml_m=rf_2,
)

In [35]:
# Run estimation
dml_plr.fit()

# Print estimation results
dml_plr.summary

c:\Python310\lib\site-packages\doubleml\_utils_checks.py:204: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=1000, random_state=0) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


,coef,std err,t,P>|t|,2.5 %,97.5 %
treatment,2.385538,0.408345,5.841972,5.158655e-09,1.585197,3.185879


In [36]:
df.groupby(['treatment'])['video_count'].count()

treatment
0    10152
1     5784
Name: video_count, dtype: int64